# Titre du projet

Notre projet vis à prédire les valeurs foncières

parser deux bases
faire la jointure entre les deux
par communes, ou à l'échelon le plus faible possible (ex : par rue)



In [3]:
import pandas as pd

FILE_PATH = "/home/onyxia/Projet-Python-2A-ENSAE/ValeursFoncieres-2025-S1.txt"

# --- LECTURE DU FICHIER ---
df = pd.read_csv(
    FILE_PATH,
    sep="|",
    dtype=str,                 # On lit tout en string pour éviter les erreurs
    decimal=",",               # Gère les valeurs foncières 468000,00
    na_values=["", " "],       # Traite les champs vides comme NaN
    engine="python"
)

# --- NETTOYAGE MINIMUM ---
# Conversion des dates
df["Date mutation"] = pd.to_datetime(df["Date mutation"], format="%d/%m/%Y", errors="coerce")

# Valeur foncière → float
df["Valeur fonciere"] = (
    df["Valeur fonciere"]
    .str.replace(",", ".", regex=False)
    .astype(float)
)

# Surface terrain
df["Surface terrain"] = pd.to_numeric(df["Surface terrain"], errors="coerce")

# Surface réelle bâtie
df["Surface reelle bati"] = pd.to_numeric(df["Surface reelle bati"], errors="coerce")

# Nombre pièces
df["Nombre pieces principales"] = pd.to_numeric(df["Nombre pieces principales"], errors="coerce")

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)
pd.set_option("display.max_rows", 20)


print(df.head())
print(df.columns)
print(f"Nombre de lignes : {len(df)}")



  Identifiant de document Reference document 1 Articles CGI 2 Articles CGI 3 Articles CGI 4 Articles CGI 5 Articles CGI No disposition Date mutation Nature mutation  Valeur fonciere No voie B/T/Q Type de voie Code voie              Voie Code postal    Commune Code departement Code commune Prefixe de section Section No plan No Volume 1er lot Surface Carrez du 1er lot 2eme lot Surface Carrez du 2eme lot 3eme lot Surface Carrez du 3eme lot 4eme lot Surface Carrez du 4eme lot 5eme lot Surface Carrez du 5eme lot Nombre de lots Code type local  Type local Identifiant local  Surface reelle bati  Nombre pieces principales Nature culture Nature culture speciale  Surface terrain
0                     NaN                NaN            NaN            NaN            NaN            NaN            NaN         000001    2025-01-07           Vente         468000.0     NaN   NaN          NaN      B078            FARGES        1550     FARGES               01          158                NaN       B     8

In [5]:
# --- CREATION D'UN DATAFRAME REDUIT ---

cols = [
    "Date mutation",
    "Valeur fonciere",
    "Surface reelle bati",
    "Surface terrain",
    "Nombre pieces principales",
    "Type local",
    "Voie",
    "Code postal",
    "Commune",
    "Code departement",
]

df2 = df[cols].copy() #on crée un autre dataframe avec un nombre réduit de colonnes
print(df2.head()) 



  Date mutation  Valeur fonciere  Surface reelle bati  Surface terrain  Nombre pieces principales  Type local              Voie Code postal    Commune Code departement
0    2025-01-07         468000.0                  NaN             78.0                        NaN         NaN            FARGES        1550     FARGES               01
1    2025-01-07         468000.0                111.0            133.0                        5.0      Maison  DE LA REPUBLIQUE        1550     FARGES               01
2    2025-01-07         468000.0                  0.0            133.0                        0.0  Dépendance  DE LA REPUBLIQUE        1550     FARGES               01
3    2025-01-06         180000.0                  NaN             46.0                        NaN         NaN        LE VILLAGE        1200  MONTANGES               01
4    2025-01-06         180000.0                  NaN             17.0                        NaN         NaN        LE VILLAGE        1200  MONTANGES          

In [6]:
df2.describe()

,Date mutation,Valeur fonciere,Surface reelle bati,Surface terrain,Nombre pieces principales
count,1387077,1.373082e+06,813890.000000,9.394530e+05,813890.000000
mean,2025-03-23 16:13:00.920453376,7.034421e+05,64.034367,3.034890e+03,1.707985
min,2025-01-01 00:00:00,1.500000e-01,0.000000,0.000000e+00,0.000000
25%,2025-02-13 00:00:00,5.935000e+04,0.000000,2.520000e+02,0.000000
50%,2025-03-24 00:00:00,1.580000e+05,28.000000,6.390000e+02,0.000000
75%,2025-04-28 00:00:00,2.950000e+05,80.000000,1.905000e+03,3.000000
max,2025-06-30 00:00:00,3.172908e+08,303623.000000,4.353772e+07,83.000000
std,NaN,4.944190e+06,635.390947,4.940377e+04,2.061545


In [ ]:
# --- NETTOYAGE BASE DE DONNEES ---

# on retire les valeurs trop petites pour les prix

df2 = df2[df2["Valeur fonciere"] > 1000]

# on retire les surfaces vendues trop petites et trop grandes pour être résidentielles (> 500m^2)

df2 = df2[(df2["Surface reelle bati"] > 10) & (df2["Surface reelle bati"] < 500)]

# de même on retire les surfaces terrain trop grandeset les autres valeurs impossibles

df2 = df2[df2["Surface terrain"] < 50000]
df2 = df2[df2["Valeur fonciere"] > 1000]
df2 = df2[df2["Surface reelle bati"] > 10]
df2 = df2[df2["Surface reelle bati"] < 500]
df2 = df2[df2["Nombre pieces principales"] <= 12]
df2 = df2[df2["Surface terrain"] < 50000]

# on prédit uniquement le prix des maisons et appartements


In [16]:
df2.describe()

,Date mutation,Valeur fonciere,Surface reelle bati,Surface terrain,Nombre pieces principales
count,258322,2.583220e+05,258322.000000,258322.000000,258322.000000
mean,2025-03-22 11:27:22.368826624,9.162719e+05,99.301295,992.079385,3.712700
min,2025-01-01 00:00:00,1.001000e+03,11.000000,0.000000,0.000000
25%,2025-02-13 00:00:00,1.310000e+05,66.000000,248.000000,3.000000
50%,2025-03-21 00:00:00,2.200000e+05,90.000000,500.000000,4.000000
75%,2025-04-26 00:00:00,3.550000e+05,120.000000,875.000000,5.000000
max,2025-06-30 00:00:00,2.496000e+08,499.000000,49767.000000,12.000000
std,NaN,6.097850e+06,55.654666,2694.630697,1.784639


In [18]:
df2["Type local"].value_counts()


Type local
Maison                                      208078
Appartement                                  34845
Local industriel. commercial ou assimilé     15399
Name: count, dtype: int64